# 2D-CNN pipeline (images ready)

In [ ]:
# ====== Imports ======
import os, random
import numpy as np
import pandas as pd
from scipy.stats import iqr
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# ====== Colab drive mount ======
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import zipfile

# Path to the zip file in Colab
zip_file_path = '/content/spider_plots.zip'

# Directory to extract to
extract_to = '/content/SPs/'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"File unzipped to {extract_to}")


File unzipped to /content/SPs/


In [ ]:
# ====== Paths ======
IMG_DIR = "/content/SPs"
CSV_PATH = "/content/train_df_averaged.csv"

TARGETS = ['Moi','NDF',  'Starch']
IMG_SIZE = 65 #------------------------------------------>16

In [ ]:
# ====== Hyperparams ======
batch_size = 128
epochs = 200
lr = 0.0001
n_splits = 5
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [ ]:
# ====== Load CSV ======
df = pd.read_csv(CSV_PATH)
# df = df.groupby('Sample ID').mean() # If the data is not averaged
# df = df.dropna(subset=TARGETS).reset_index(drop=True)
print("Data shape:", df.shape)

Data shape: (784, 261)


In [ ]:
# ====== Scale targets ======
from sklearn.preprocessing import StandardScaler
# y_scaler = MinMaxScaler()
y_scaler = StandardScaler() # => Auto scale
targets_scaled = y_scaler.fit_transform(df[TARGETS].values.astype(np.float32))

In [ ]:
plot_name = "spider_plot"  # Change this based on the plot type

In [ ]:
# ====== Dataset class (images already ready) ======

class SoilImageDataset(Dataset):
    def __init__(self, df, img_dir, targets_scaled, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.targets = targets_scaled.astype(np.float32)
        self.transform = transform if transform else transforms.ToTensor()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, f"{plot_name}_{idx}.png")
        img = Image.open(img_path).resize((IMG_SIZE, IMG_SIZE))  # grayscale
        img = self.transform(img)  # shape (1,H,W)
        target = torch.from_numpy(self.targets[idx])
        return img, target

transform = transforms.Compose([
    transforms.ToTensor(),  # (H,W) -> (1,H,W), float in [0,1]
])

dataset = SoilImageDataset(df, IMG_DIR, targets_scaled, transform=transform)
print("Dataset length:", len(dataset))

Dataset length: 784


I`m gonna experiment 1st with Grayscale on CV

#Then RGB

In [ ]:
# ====== CNN model (as in Table 2) ======
class CNN2D(nn.Module):
    def __init__(self, in_channels=3, num_outputs=3): # Changed num_outputs to 3
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3a = nn.Conv2d(128, 256, 3, padding=1)
        self.conv3b = nn.Conv2d(256, 256, 3, padding=1)
        self.conv4a = nn.Conv2d(256, 512, 3, padding=1)
        self.conv4b = nn.Conv2d(512, 512, 3, padding=1)
        self.conv5a = nn.Conv2d(512, 512, 3, padding=1)
        self.conv5b = nn.Conv2d(512, 512, 3, padding=1)

        self.flattened = 512 * 2 * 2
        self.fc1 = nn.Linear(self.flattened, 128)
        self.fc2 = nn.Linear(128, num_outputs)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x)); x = self.pool(x)
        x = self.relu(self.conv2(x)); x = self.pool(x)
        x = self.relu(self.conv3a(x)); x = self.relu(self.conv3b(x)); x = self.pool(x)
        x = self.relu(self.conv4a(x)); x = self.relu(self.conv4b(x)); x = self.pool(x)
        x = self.relu(self.conv5a(x)); x = self.relu(self.conv5b(x)); x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
%pip install torchsummary

In [ ]:
# ====== Model Summary ======
from torchsummary import summary
# Instantiate the model
model_summary = CNN2D().to(device)

# Print the model summary
print("Model Summary:")
summary(model_summary, input_size=(3, IMG_SIZE, IMG_SIZE)) # Assuming input is grayscale (1 channel)

# Print the number of parameters
num_params = sum(p.numel() for p in model_summary.parameters() if p.requires_grad)
print(f"\nNumber of parameters: {num_params}")

Model Summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 65, 65]           1,792
              ReLU-2           [-1, 64, 65, 65]               0
         MaxPool2d-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]          73,856
              ReLU-5          [-1, 128, 32, 32]               0
         MaxPool2d-6          [-1, 128, 16, 16]               0
            Conv2d-7          [-1, 256, 16, 16]         295,168
              ReLU-8          [-1, 256, 16, 16]               0
            Conv2d-9          [-1, 256, 16, 16]         590,080
             ReLU-10          [-1, 256, 16, 16]               0
        MaxPool2d-11            [-1, 256, 8, 8]               0
           Conv2d-12            [-1, 512, 8, 8]       1,180,160
             ReLU-13            [-1, 512, 8, 8]               0
           Conv2d-14    

In [ ]:
# ====== Metrics ======
def compute_metrics_orig(y_true, y_pred):
    results = []
    for i in range(y_true.shape[1]):
        yt, yp = y_true[:, i], y_pred[:, i]
        rmse = np.sqrt(mean_squared_error(yt, yp))
        r2 = r2_score(yt, yp)
        rpiq = float(iqr(yt) / rmse) if rmse > 1e-8 else float("inf")
        results.append({"RMSE": rmse, "R2": r2, "RPIQ": rpiq})
    return results

In [ ]:
# ====== Split train/test ======
indices = np.arange(len(dataset))
trainval_idx, test_idx = train_test_split(indices, test_size=0.25, random_state=seed)

In [ ]:
# ====== 5-Fold CV ====== But more Verbose
kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
for fold, (t_idx, v_idx) in enumerate(kf.split(trainval_idx)):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")
    train_loader = DataLoader(Subset(dataset, trainval_idx[t_idx]), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(Subset(dataset, trainval_idx[v_idx]), batch_size=batch_size, shuffle=False)

    model = CNN2D().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()

        # Verbose mode
        if (epoch+1) % 1 == 0:  # This will print every epoch (verbose=2 equivalent in training)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

    # Validation
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            preds = model(xb.to(device)).cpu().numpy()
            yb_np = yb.cpu().numpy()
            preds_orig = y_scaler.inverse_transform(preds)
            yb_orig = y_scaler.inverse_transform(yb_np)
            y_true.append(yb_orig); y_pred.append(preds_orig)
    y_true, y_pred = np.vstack(y_true), np.vstack(y_pred)

    # Compute and print metrics
    metrics = compute_metrics_orig(y_true, y_pred)
    for i, t in enumerate(TARGETS):
        print(f"{t}: RMSE={metrics[i]['RMSE']:.3f}, R2={metrics[i]['R2']:.3f}, RPIQ={metrics[i]['RPIQ']:.3f}")



--- Fold 1/5 ---
Epoch 1/100, Loss: 1.1221
Epoch 2/100, Loss: 0.8388
Epoch 3/100, Loss: 1.1384
Epoch 4/100, Loss: 0.9702
Epoch 5/100, Loss: 0.9403
Epoch 6/100, Loss: 0.8286
Epoch 7/100, Loss: 1.0344
Epoch 8/100, Loss: 1.0219
Epoch 9/100, Loss: 0.8753
Epoch 10/100, Loss: 1.4952
Epoch 11/100, Loss: 1.0310
Epoch 12/100, Loss: 0.7568
Epoch 13/100, Loss: 1.2121
Epoch 14/100, Loss: 0.9335
Epoch 15/100, Loss: 0.9789
Epoch 16/100, Loss: 0.9108
Epoch 17/100, Loss: 0.9121
Epoch 18/100, Loss: 1.0057
Epoch 19/100, Loss: 1.0687
Epoch 20/100, Loss: 0.7762
Epoch 21/100, Loss: 0.7351
Epoch 22/100, Loss: 0.7385
Epoch 23/100, Loss: 0.8654
Epoch 24/100, Loss: 0.6992
Epoch 25/100, Loss: 0.9706
Epoch 26/100, Loss: 0.6838
Epoch 27/100, Loss: 0.9048
Epoch 28/100, Loss: 1.0692
Epoch 29/100, Loss: 0.6192
Epoch 30/100, Loss: 0.9026
Epoch 31/100, Loss: 0.8715
Epoch 32/100, Loss: 0.8911
Epoch 33/100, Loss: 0.8515
Epoch 34/100, Loss: 0.8059
Epoch 35/100, Loss: 0.9232
Epoch 36/100, Loss: 1.0161
Epoch 37/100, Loss:

In [ ]:
# ====== Calculate mean of the metrics obtained in each fold in CV ======

# Lists to store metrics for each fold
all_fold_metrics = {target: {"RMSE": [], "R2": [], "RPIQ": []} for target in TARGETS}

# ... (rest of the code from the previous cell for the CV loop)
# After computing metrics for each fold:
for i, t in enumerate(TARGETS):
     print(f"{t}: RMSE={metrics[i]['RMSE']:.3f}, R2={metrics[i]['R2']:.3f}, RPIQ={metrics[i]['RPIQ']:.3f}")
     all_fold_metrics[t]["RMSE"].append(metrics[i]["RMSE"])
     all_fold_metrics[t]["R2"].append(metrics[i]["R2"])
     all_fold_metrics[t]["RPIQ"].append(metrics[i]["RPIQ"])


Moi: RMSE=2.735, R2=0.815, RPIQ=2.601
NDF: RMSE=4.073, R2=0.609, RPIQ=1.719
Starch: RMSE=6.686, R2=0.620, RPIQ=1.538


In [ ]:
# ====== Train final on all trainval + test evaluation ======
# Create data loaders for training and testing
final_loader = DataLoader(Subset(dataset, trainval_idx), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(Subset(dataset, test_idx), batch_size=batch_size, shuffle=False)

# Initialize the model, optimizer, and loss function
final_model = CNN2D().to(device)  # Move model to the specified device (GPU or CPU)
optimizer = torch.optim.Adam(final_model.parameters(), lr=lr)  # Adam optimizer
criterion = nn.MSELoss()  # Mean Squared Error loss for regression tasks

# Start training for a specified number of epochs
for epoch in range(epochs):
    final_model.train()  # Set the model to training mode (enables dropout, batchnorm, etc.)

    # Initialize variables to track loss during this epoch
    epoch_loss = 0.0  # Track cumulative loss for the current epoch

    # Iterate over the batches in the training dataset
    for xb, yb in final_loader:
        xb, yb = xb.to(device), yb.to(device)  # Move data to the specified device (GPU or CPU)

        optimizer.zero_grad()  # Clear the previous gradients
        loss = criterion(final_model(xb), yb)  # Compute the loss for this batch
        loss.backward()  # Backpropagate the gradients
        optimizer.step()  # Update the model's parameters

        epoch_loss += loss.item()  # Accumulate loss for this batch

    # Print progress after each epoch
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {epoch_loss / len(final_loader):.4f}")

# Evaluate the model on the test dataset
final_model.eval()  # Set the model to evaluation mode (disables dropout, batchnorm)
y_true, y_pred = [], []  # Initialize lists to store true and predicted values

with torch.no_grad():  # Disable gradient computation for inference
    for xb, yb in test_loader:
        preds = final_model(xb.to(device)).cpu().numpy()  # Get predictions from the model
        yb_np = yb.cpu().numpy()  # Get true values from the batch

        # Reverse scaling (if applicable) to get the original values
        preds_orig = y_scaler.inverse_transform(preds)
        yb_orig = y_scaler.inverse_transform(yb_np)

        y_true.append(yb_orig)  # Append true values to the list
        y_pred.append(preds_orig)  # Append predicted values to the list

# Convert the lists to numpy arrays for metric computation
y_true, y_pred = np.vstack(y_true), np.vstack(y_pred)

# Compute the evaluation metrics (RMSE, R2, RPIQ) for the test set
test_metrics = compute_metrics_orig(y_true, y_pred)

# Print the metrics for each target
print("\n== Test metrics ==")
for i, t in enumerate(TARGETS):
    print(f"{t}: RMSE={test_metrics[i]['RMSE']:.3f}, R2={test_metrics[i]['R2']:.3f}, RPIQ={test_metrics[i]['RPIQ']:.3f}")

# Save the trained model to a file with a dynamic name
file_path = f"/content/{plot_name}_images_ready.pth"
torch.save(final_model.state_dict(), file_path)  # Save the model's state dictionary
print(f"Saved model to {file_path}")


Epoch 1/200, Train Loss: 0.9758
Epoch 2/200, Train Loss: 0.9684
Epoch 3/200, Train Loss: 0.9658
Epoch 4/200, Train Loss: 0.9773
Epoch 5/200, Train Loss: 0.9620
Epoch 6/200, Train Loss: 0.9657
Epoch 7/200, Train Loss: 0.9758
Epoch 8/200, Train Loss: 0.9692
Epoch 9/200, Train Loss: 0.9649
Epoch 10/200, Train Loss: 0.9577
Epoch 11/200, Train Loss: 0.9759
Epoch 12/200, Train Loss: 0.9632
Epoch 13/200, Train Loss: 0.9937
Epoch 14/200, Train Loss: 0.9750
Epoch 15/200, Train Loss: 0.9503
Epoch 16/200, Train Loss: 0.9830
Epoch 17/200, Train Loss: 0.9572
Epoch 18/200, Train Loss: 0.9711
Epoch 19/200, Train Loss: 0.9621
Epoch 20/200, Train Loss: 0.9838
Epoch 21/200, Train Loss: 0.9363
Epoch 22/200, Train Loss: 0.8992
Epoch 23/200, Train Loss: 0.8715
Epoch 24/200, Train Loss: 0.8844
Epoch 25/200, Train Loss: 0.8786
Epoch 26/200, Train Loss: 0.8548
Epoch 27/200, Train Loss: 0.8296
Epoch 28/200, Train Loss: 0.8058
Epoch 29/200, Train Loss: 0.8109
Epoch 30/200, Train Loss: 0.7901
Epoch 31/200, Train